In [1]:
import os
import json
import random
from typing import List, Dict

def write_jsonl_output(directory: str, output_file: str) -> None:
    """
    遍历指定目录及其所有子目录中的 JSON 文件，每个子文件夹随机选择 10 个 JSON 文件，
    并将其信息以 JSON Lines 格式写入到指定的输出文件中。
    
    参数:
    directory (str): JSON 文件所在的根目录路径
    output_file (str): 输出的 JSON Lines 文件路径
    """
    try:
        with open(output_file, 'w', encoding='utf-8') as outfile:
            # 使用 os.walk() 遍历根目录及其所有子目录
            for root, dirs, files in os.walk(directory):
                # 只处理 .json 结尾的文件
                json_files = [file for file in files if file.endswith('.json')]
                
                # 如果 json 文件数量大于 10，随机选择 10 个文件
                if len(json_files) > 10:
                    selected_files = random.sample(json_files, 10)
                else:
                    selected_files = json_files

                # 处理每个被选中的 JSON 文件
                for json_file in selected_files:
                    json_path = os.path.join(root, json_file)
                    
                    try:
                        with open(json_path, 'r', encoding='utf-8') as file:
                            content = json.load(file)

                            # 提取字段
                            folder_name = os.path.basename(root)
                            file_path = json_path
                            tool_name = content.get('name', '')
                            description = content.get('tool_description', '')

                            # 写入 JSON Lines 格式
                            json_line = {
                                'folder_name': folder_name,
                                'file_path': file_path,
                                'tool_name': tool_name,
                                'description': description
                            }
                            outfile.write(json.dumps(json_line, ensure_ascii=False) + '\n')
                    
                    except json.JSONDecodeError as json_error:
                        print(f"文件解析失败: {json_file}, 错误: {json_error}")
                    except FileNotFoundError as fnf_error:
                        print(f"文件未找到: {fnf_error}")
                    except Exception as e:
                        print(f"处理文件 {json_file} 时出错: {e}")

    except FileNotFoundError as fnf_error:
        print(f"目录未找到: {fnf_error}")
    except PermissionError as perm_error:
        print(f"权限不足: {perm_error}")
    except Exception as e:
        print(f"出现错误: {e}")

def load_json_from_jsonl(jsonl_file: str) -> List[Dict]:
    """
    读取 JSON Lines 文件，根据每个记录中的 'file_path' 字段读取对应的 JSON 文件，
    并返回所有 JSON 文件的内容对象列表。
    
    参数:
    jsonl_file (str): 包含 JSON 对象的 JSON Lines 文件路径
    
    返回:
    List[Dict]: 包含所有 JSON 文件内容对象的列表
    """
    json_objects = []

    try:
        with open(jsonl_file, 'r', encoding='utf-8') as infile:
            for line in infile:
                record = json.loads(line)
                file_path = record.get('file_path', '')
                
                if file_path:
                    try:
                        with open(file_path, 'r', encoding='utf-8') as json_file:
                            content = json.load(json_file)
                            json_objects.append(content)
                    
                    except json.JSONDecodeError as json_error:
                        print(f"文件解析失败: {file_path}, 错误: {json_error}")
                    except FileNotFoundError as fnf_error:
                        print(f"文件未找到: {fnf_error}")
                    except Exception as e:
                        print(f"处理文件 {file_path} 时出错: {e}")

    except FileNotFoundError as fnf_error:
        print(f"文件未找到: {fnf_error}")
    except PermissionError as perm_error:
        print(f"权限不足: {perm_error}")
    except Exception as e:
        print(f"出现错误: {e}")

    return json_objects

# 你要遍历的文件夹路径
directory_path = "/Users/caizhuoyue/Desktop/热血毕业/RapidAPIHub-sample/tools"
# 输出的 JSON Lines 文件路径
output_file_path = "/Users/caizhuoyue/Desktop/热血毕业/RapidAPIHub-sample/output.jsonl"

# 调用函数，随机选择每个子文件夹中的 10 个 JSON 文件，并输出到 JSON Lines 文件
write_jsonl_output(directory_path, output_file_path)

# 根据 JSON Lines 文件读取所有 JSON 文件的内容
json_objects = load_json_from_jsonl(output_file_path)
print(json_objects[:2])

[{'product_id': 'api_359d13fb-c997-41c6-9dd2-b2da7cbae73b', 'tool_description': 'API for songs', 'home_url': 'https://rapidapi.com/chanchal172002/api/musiczone1/', 'name': 'MusicZone', 'title': 'MusicZone', 'pricing': 'FREE', 'tool_name': 'MusicZone', 'score': {'avgServiceLevel': 0, 'avgLatency': 697, 'avgSuccessRate': 0, 'popularityScore': 0, '__typename': 'Score'}, 'host': 'musiczone1.p.rapidapi.com', 'api_list': [{'name': 'getSongs', 'url': 'https://musiczone1.p.rapidapi.com/', 'description': 'get song details', 'method': 'GET', 'required_parameters': [], 'optional_parameters': [], 'code': 'import requests\n\nurl = "https://musiczone1.p.rapidapi.com/"\n\nheaders = {\n            "X-RapidAPI-Key": "SIGN-UP-FOR-KEY",\n            "X-RapidAPI-Host": "musiczone1.p.rapidapi.com"\n        }\n\nresponse = requests.get(url, headers=headers)\nprint(response.json())\n', 'convert_code': 'import requests\n\nurl = "https://musiczone1.p.rapidapi.com/"\n\nheaders = {\n            "X-RapidAPI-Key":

In [2]:
from openai import OpenAI
from typing import Optional

def get_chat_response(prompt: str) -> Optional[str]:
    """
    使用 OpenAI API 获取聊天回复。

    参数:
    prompt (str): 要发送到聊天模型的提示语

    返回:
    Optional[str]: 模型生成的回复内容。如果发生错误，返回 None。
    """
    # 初始化 OpenAI 客户端
    try:
        client = OpenAI(
            api_key="sk-H0UFSY3LvO8YBt092HnMfLO6oRHZjbwdwTVucnxRwVEUzZAA",
            base_url="https://api.chatanywhere.tech/v1"
            # base_url="https://api.chatanywhere.cn/v1"
        )
    except Exception as e:
        print(f"初始化 OpenAI 客户端时出错: {e}")
        return None

    # 调用 chat.completions.create 方法
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
    except Exception as e:
        print(f"调用 OpenAI API 时出错: {e}")
        return None

    # 提取并返回 response 中的 content 字段
    try:
        content = response.choices[0].message.content if response.choices else "No content available"
        return content
    except AttributeError as e:
        print(f"解析 API 响应时出错: {e}")
        return None

In [10]:
api_info = f"{json_objects[0]['name']}:{json_objects[0]['tool_description']}"
user_prompt = f"tell me if this is the right choice for getting weather information. Simple 'yes' or 'no' will do. API: {api_info}"


GET_NODE_PROMPT = ""
GET_EDGE_PROMPT = ""

for tool in json_objects: # 遍历所有的工具
    if "api_list" in tool: # 遍历所有的api
        # 需要详细的prompt设计
        
        print(tool["api_list"][0])
    
        response_content = get_chat_response(user_prompt)

if response_content is not None:
    print(response_content)
else:
    print("获取响应失败。")

{'name': 'getSongs', 'url': 'https://musiczone1.p.rapidapi.com/', 'description': 'get song details', 'method': 'GET', 'required_parameters': [], 'optional_parameters': [], 'code': 'import requests\n\nurl = "https://musiczone1.p.rapidapi.com/"\n\nheaders = {\n            "X-RapidAPI-Key": "SIGN-UP-FOR-KEY",\n            "X-RapidAPI-Host": "musiczone1.p.rapidapi.com"\n        }\n\nresponse = requests.get(url, headers=headers)\nprint(response.json())\n', 'convert_code': 'import requests\n\nurl = "https://musiczone1.p.rapidapi.com/"\n\nheaders = {\n            "X-RapidAPI-Key": "SIGN-UP-FOR-KEY",\n            "X-RapidAPI-Host": "musiczone1.p.rapidapi.com"\n        }\n\nresponse = requests.get(url, headers=headers)\nprint(response.json())\n', 'test_endpoint': ''}
{'name': 'Get Cities', 'url': 'https://50k-radio-stations.p.rapidapi.com/get/cities', 'description': 'Get city list', 'method': 'GET', 'required_parameters': [], 'optional_parameters': [{'name': 'country_id', 'type': 'NUMBER', 'des